# 🐟 SaRDinE Training Notebook

**Sparse Routed Delta Experts on Mistral-14B-Reasoning**

This notebook trains SRDE with:
- W&B logging
- Auto-upload to HuggingFace
- Ablation support

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets accelerate wandb huggingface_hub bitsandbytes

In [ ]:
# Clone the SRDE repo
!git clone https://github.com/MinimaML/srde-mistral
%cd srde-mistral/mistral-3-14b-reasoning

In [ ]:
# Login to W&B (paste your API key when prompted)
import wandb
wandb.login()

In [ ]:
# Login to HuggingFace (for auto-upload)
from huggingface_hub import login
login()  # Paste your HF token when prompted

## 2. Configuration

In [ ]:
# Training config - edit these!
CONFIG = {
    # W&B
    "wandb_project": "sardine-collab",
    "wandb_run_name": "sardine-b200-run",
    
    # HuggingFace upload
    "hf_repo": "MinimaML/SaRDinE-14B8x4P",
    
    # Training
    "num_epochs": 3,
    "max_steps": 100000,
    "batch_size": 4,  # Increase for B200!
    "gradient_accumulation_steps": 4,
    "learning_rate": 1e-4,
    "save_steps": 2500,
    
    # SRDE
    "num_experts": 8,
    "target_sparsity": 0.01,
    
    # Ablation (set to None for normal run)
    "ablation_name": None,  # e.g., "4_experts", "no_router"
}

## 3. Load Model

In [ ]:
import torch
from transformers import AutoTokenizer
from srde import create_srde_model
from config import SRDEConfig

print("Loading model...")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Ministral-3-14B-Reasoning-2512",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

# SRDE Model
model = create_srde_model(
    model_name="mistralai/Ministral-3-14B-Reasoning-2512",
    torch_dtype=torch.bfloat16
)

print(f"Trainable params: {model.trainable_param_count() / 1e9:.2f}B")

## 4. Load Data

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader

# Load dataset
dataset = load_dataset("MinimaML/srde-reasoning-mix", split="train")

def tokenize(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding="max_length"
    )

dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)
dataset.set_format("torch")

dataloader = DataLoader(
    dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,
    num_workers=4
)

print(f"Dataset size: {len(dataset)}")

## 5. Setup Training

In [ ]:
from datetime import datetime

# W&B init
run_name = CONFIG["wandb_run_name"] or f"sardine-{datetime.now().strftime('%m%d-%H%M')}"
if CONFIG["ablation_name"]:
    run_name = f"ablation-{CONFIG['ablation_name']}"

wandb.init(
    project=CONFIG["wandb_project"],
    name=run_name,
    config=CONFIG
)

# Optimizer
optimizer = torch.optim.AdamW(
    model.get_trainable_params(),
    lr=CONFIG["learning_rate"],
    weight_decay=0.01
)

print("Training setup complete!")

## 6. Train!

In [ ]:
from tqdm.auto import tqdm
from pathlib import Path

device = "cuda"
model.train()
global_step = 0
best_loss = float('inf')

output_dir = Path("./checkpoints")
output_dir.mkdir(exist_ok=True)

for epoch in range(CONFIG["num_epochs"]):
    print(f"\n=== Epoch {epoch} ===")
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}")
    
    for batch_idx, batch in enumerate(pbar):
        if global_step >= CONFIG["max_steps"]:
            break
            
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        # Forward
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss / CONFIG["gradient_accumulation_steps"]
        
        # Backward
        loss.backward()
        
        # Update
        if (batch_idx + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
            current_loss = loss.item() * CONFIG["gradient_accumulation_steps"]
            
            # Log to W&B
            if global_step % 10 == 0:
                wandb.log({"loss": current_loss, "step": global_step})
                pbar.set_postfix({"loss": f"{current_loss:.4f}", "step": global_step})
            
            # Save checkpoint
            if global_step % CONFIG["save_steps"] == 0:
                save_path = output_dir / f"checkpoint-{global_step}"
                save_path.mkdir(exist_ok=True)
                model.save_srde_weights(str(save_path / "srde_weights.pt"))
                print(f"\nSaved checkpoint to {save_path}")
                
                if current_loss < best_loss:
                    best_loss = current_loss
                    best_path = output_dir / "checkpoint-best"
                    best_path.mkdir(exist_ok=True)
                    model.save_srde_weights(str(best_path / "srde_weights.pt"))

print(f"\nTraining complete! Final step: {global_step}")

## 7. Upload to HuggingFace

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Upload best checkpoint
api.upload_folder(
    folder_path="./checkpoints/checkpoint-best",
    repo_id=CONFIG["hf_repo"],
    commit_message=f"Training complete - {global_step} steps - loss {best_loss:.4f}"
)

print(f"Uploaded to: https://huggingface.co/{CONFIG['hf_repo']}")

In [ ]:
# Finish W&B
wandb.finish()
print("Done! 🐟")